In [20]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [57]:
### Load the trained model, scaler pickle,onehot
model=load_model('model.h5')

## load the encoder and scaler
with open('lable_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('Scalar.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [58]:
# Example input data
input_data = {
    'CreditScore': 765,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 35,
    'Tenure': 10,
    'Balance': 600000,
    'NumOfProducts': 3,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 100000
}

In [59]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/santoshdevarasetty/Downloads/PythonProject/Udemy_langchainGenAI/myenv312/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [60]:
inputdf=pd.DataFrame([input_data])
inputdf

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,765,France,Male,35,10,600000,3,1,1,100000


In [61]:
inputdf=pd.concat([inputdf.drop("Geography",axis=1),geo_encoded_df],axis=1)
inputdf

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,765,Male,35,10,600000,3,1,1,100000,1.0,0.0,0.0


In [62]:
inputdf['Gender']=label_encoder_gender.transform(inputdf['Gender'])
inputdf

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,765,1,35,10,600000,3,1,1,100000,1.0,0.0,0.0


In [63]:
input_scalar = scaler.transform(inputdf)
input_scalar

array([[ 1.1673175 ,  0.90240414, -0.3694588 ,  1.7368167 ,  8.37963798,
         2.5541237 ,  0.64583254,  0.96430337, -0.02073069,  0.9943493 ,
        -0.56965192, -0.5806861 ]])

In [64]:
##predict
get_my_prediction = model.predict(input_scalar)
get_my_prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


array([[0.43504822]], dtype=float32)

In [65]:
predication_proba = get_my_prediction[0][0]
predication_proba

0.43504822

In [66]:
if predication_proba>0.5:
    print("The Customer likely to churn")
else:
    print("The Customer not likely to churn")

The Customer not likely to churn
